In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import Compose, Normalize, ToTensor, Resize, CenterCrop
import matplotlib.pyplot as plt
from tqdm import tqdm
from typing import Optional, cast
import wandb
from models.dino_icarl import DinoIcarlModel
from utilities.wandb_utils import load_checkpoint_from_wandb, save_checkpoint_to_wandb

ENTITY = "aml-fl-project"
PROJECT = "fl-task-arithmetic"
GROUP = "centralized-dino-cifar100"

BATCH_SIZE = 64
LR  = 0.01           # Learning Rate (Tune this: 0.1, 0.01, 0.001)
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-4
EPOCHS = 20         # Increase to 100+ for final results
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Standard CIFAR-100 Normalization
stats = ((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))

# Transforms
transform_train = Compose([
    Resize(256), CenterCrop(224), # Required for DINO
    # transforms.RandomHorizontalFlip(), # Optional augmentation
    ToTensor(),
    Normalize(*stats),
])

transform_test = Compose([
    Resize(256), CenterCrop(224),
    ToTensor(),
    Normalize(*stats),
])

trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

def train(lr, momentum, weight_decay, epochs):
    run_id = f"run-1-centralized-dino-icarl-cifar100-lr{lr}-mom{momentum}-wd{weight_decay}"
    run = wandb.init(
        entity=ENTITY,
        project=PROJECT,
        group=GROUP,
        name=f"centralized-dino-icarl-cifar100-lr{lr}-mom{momentum}-wd{weight_decay}",
        id=run_id,
        resume="allow",
        mode="online",
    )

    model = DinoIcarlModel(device=DEVICE)

    checkpoint = load_checkpoint_from_wandb(
        run,
        model,
        "model.pth"
    )
    start_epoch = 0
    if checkpoint is not None:
        checkpoint_dict, artifact = checkpoint
        model.load_state_dict(checkpoint_dict['model'])
        start_epoch = artifact.metadata["epoch"] + 1
        print(f"Resuming from epoch {start_epoch}")
    else:
        print("Starting from scratch")

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
    train_losses = []
    test_losses = []
    test_accs = []

    for epoch in range(start_epoch, EPOCHS):
        model.train()
        running_loss = 0.0

        # Progress bar for training
        pbar = tqdm(trainloader, desc=f"Epoch {epoch+1}/{EPOCHS}")
        for images, labels in pbar:
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix({'loss': loss.item()})

        scheduler.step()

        avg_train_loss = running_loss / len(trainloader)
        train_losses.append(avg_train_loss)

        # 5. Evaluation
        model.eval()
        test_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in testloader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = model(images)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        avg_test_loss = test_loss / len(testloader)
        acc = 100. * correct / total

        test_losses.append(avg_test_loss)
        test_accs.append(acc)

        print(f"Epoch {epoch+1} Results: Train Loss: {avg_train_loss:.4f} | Test Loss: {avg_test_loss:.4f} | Test Acc: {acc:.2f}%")


        save_checkpoint_to_wandb(run, {
            'model': model.state_dict(),
        }, f"model.pth", {
            "task": model,
            "accuracy": acc,
        })
        print(epoch, "Saved checkpoint model to WandB.")

    
train(LR, MOMENTUM, WEIGHT_DECAY, EPOCHS)



wandb: Currently logged in as: adrientrahan (aml-fl-project) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Using cache found in /Users/adrientrahan/.cache/torch/hub/facebookresearch_dino_main


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Using cache found in /Users/adrientrahan/.cache/torch/hub/facebookresearch_dino_main
Using cache found in /Users/adrientrahan/.cache/torch/hub/facebookresearch_dino_main
wandb: Downloading large artifact 'icarl-cifar100-checkpoints:latest', 194.41MB. 1 files...
wandb:   1 of 1 files downloaded.  
Done. 00:00:00.5 (367.7MB/s)


Loading model from: /Users/adrientrahan/Documents/ecole/AML/project/FL-task-arithmetic/notebooks/artifacts/icarl-cifar100-checkpoints:v2/model.pth
Successfully loaded model from: /Users/adrientrahan/Documents/ecole/AML/project/FL-task-arithmetic/notebooks/artifacts/icarl-cifar100-checkpoints:v2/model.pth
Run (run-1-centralized-dino-icarl-cifar100-lr0.01-mom0.9-wd0.0005) is finished. The call to `use_artifact` will be ignored. Please make sure that you are using an active run.
Model checkpoint not found on WandB. Run (run-1-centralized-dino-icarl-cifar100-lr0.01-mom0.9-wd0.0005) is finished. The call to `use_artifact` will be ignored. Please make sure that you are using an active run.
Starting from scratch


Epoch 1/20:   1%|          | 4/782 [00:19<1:03:22,  4.89s/it, loss=98.4]
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.


KeyboardInterrupt: 

socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.


socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.s